### Cora dataset ideas:


- Feature type: Bag-of-Words (binary)

- Each node has a 1433-dimensional feature vector

- Each dimension corresponds to a unique word in the vocabulary

- Values are binary:

- 1 : the word appers in the paper

- 0 : the word does not appear

### 1. Node Classification
   - Thousands of papers are published every week —> manual categorization doesn’t scale.

  #### plan:
  - apply Shallow Embeddings with Node2Vec (only uses graph structure 'no features') to show classifiction performance only utilizing citations links
  - apply Deep Embeddings (GNNs) like GCN and GAT (makes use of node features) and perform classifiction
  - analysis (compare methods accuracies , t-SNE plots of embeddings colored by class (for each method),Discuss trade-offs (scalability, expressiveness, transductive vs. inductive))



### 2. Link Prediction
  - Authors often miss relevant prior work.
  - Citation recommendation during writing
  - Related-work discovery
  - Recommending papers that an author should read or cite based on the graph structure and/or content similarity.

  #### plan:
  - edge splitting and generating negative samples 
  - learn embeddings with link prediction objective (not sure if node2vec can use same embeddings generated for node classification or not, but for gnn we need new embeddings cause loss is different)
  - check scoring methods (For each pair (u, v) in the test set, compute a score using the embeddings using Dot product/Cosine similarity/mlp classifier )
  - evaluate predictions 



### Twitch dataset ideas:

There are 5 languages, English, French, German, Italian, and Spanish. Each language community is disconnected from the others. Each node has the following features:
id,days,mature,views,partner,new_id
### 1. Node Classification
   - Automatically flagging streamers who use explicit language —> manual review doesn't scale across 32K+ streamers and multiple languages

  #### plan:
  - Approach 1: Per-language models (baseline):
    -   train and evaluate within each language separately:
    - For each language graph:
        - Node2Vec → logistic regression (structure only, no features)
        - GCN / GAT (structure + features, end-to-end)
        - Stratified train/val/test split within that language
    - Compare: does graph structure actually help? Are some languages easier to classify than others? Does graph density (DE is dense, ENGB is sparse) affect GNN performance?
- Approach 2: Transfer learning across languages
    - All languages share the same feature space:
    - Train a GCN/GAT on one language → test on a different language (zero-shot transfer)
    - So, we train on DE (largest, densest) → test on RU (smallest)
    - This tests whether the relationship between gaming preferences, social connections, and explicit language use is language/culture-invariant
    Compare transfer accuracy across all language pairs,  which transfers well, which doesn't?
    - create a Transfer Matrix: 
        - Diagonal = train and test on the same language (normal in-domain performance, the upper bound)
        - Off-diagonal = transfer performance (train on row, test on column)
        - For each cell in the matrix: Accuracy, F1-score, AUC-ROC (since it's binary classification and mature may be imbalanced)
    - we analyze: 
        - How much does transfer performance drop vs. in-domain? If off-diagonal scores are close to diagonal → explicit language behavior follows universal social patterns across cultures. If there's a big drop → the relationship between features/structure and mature is culture-specific
        - Which language transfers best as a source?
        - Are some language pairs more similar? use a heatmap
    - Per-language accuracy/F1/AUC-ROC comparison table
t-SNE of embeddings colored by (a) mature and (b) language — do language communities cluster separately even without explicit language features?
Feature-only baseline (no graph, just logistic regression on features) — how much does the graph add?
Transfer learning matrix: train on row language, test on column language




### 2. Link Prediction
  - which streamers would be friends based on their gaming preferences and existing social ties
  - Since there are no cross-language edges, link prediction only makes sense within each language. We should:
- Split edges per language (e.g., 85/5/10 train/val/test)
- Generate negative samples only from same-language node pairs (sampling random pairs across the full graph would be trivially easy since cross-language edges don't exist)
- Ensure the training graph stays connected within each component after edge removal

#### plan:
  - Node2Vec: Embed on the training subgraph per language, score test edges via dot product / cosine similarity
- GCN/GAT with link prediction head: Train with binary cross-entropy on edge existence — separate from node classification since the loss and training signal are fundamentally different
- Scoring: For each candidate pair (u, v), try dot product, cosine similarity, and Hadamard product → MLP
- Evaluation
    - AUC-ROC and Average Precision per language
    - Compare: do denser networks (DE, FR) have better link prediction than sparser ones (ENGB, RU)? Denser graphs have more training signal but also more possible negatives.
    - Does including node features improve link prediction over structure-only methods? (i.e., are streamers friends because they like the same games, or for other reasons?)
